In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline 
sns.set_style("white")

In [ ]:
## select the dataset
df=pd.read_csv('../Final/Berlin/merge_data_files/Entire_home_apt.csv' ,low_memory=False,index_col=0) 


In [ ]:
# Listings segmentation using K-Means Clustering


In [ ]:
pd.set_option("display.max_columns", 30) 
df.head()

In [ ]:
pd.get_option("display.max_rows")
df.isnull().sum()

In [ ]:
pd.set_option("display.max_rows", 80) 
df.nunique()

In [ ]:
df.price

In [ ]:


df=df.loc[df.price<1000]

In [ ]:
df

In [ ]:
# find the the mean for each neighborhood

df_mean=df.groupby(['neighbourhood_cleansed', 'date'],as_index=False).agg({'price': 'median', 'id' : 'count'})
df_mean

In [ ]:
df=pd.merge(df, df_mean, on=['neighbourhood_cleansed','date'], how='left')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
X = df[["latitude", "longitude", 'calculated_host_listings_count']]
        #,'price','number_of_reviews']]


In [ ]:
###dont run

inertias = {}

for k in range(1,15):
    cl = KMeans(n_clusters=k).fit(X)
    inertias[k] = cl.inertia_
plt.figure(figsize=(10,6))
sns.lineplot(x=list(inertias.keys()),
             y=list(inertias.values()));
plt.title("Optimal number of clusters");

In [ ]:
## model

clusters = KMeans(n_clusters=3)
clusters.fit(X)
data_cluster=X.assign(cluster=clusters.predict(X))
data_cluster

In [ ]:

sns.scatterplot(data=X,
                x="longitude",
                y="latitude", 
                hue=clusters.predict(X));

In [ ]:
df=pd.concat([df, data_cluster['cluster'].astype(str)], axis=1)

In [ ]:
df=df.assign(diff=(df['price_x']-df['price_y'])/df['price_y']+1)

In [ ]:
df

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", color='cluster',
                        size='diff',
                        hover_name="host_name",
                        hover_data=['listing_url',"host_id","calculated_host_listings_count",'neighbourhood_cleansed',
                                    'price_x','price_y'],
                        color_discrete_sequence=['blue','black','orange'],
                       
                        animation_frame="date",
                        zoom=10, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
import plotly.graph_objs as go
scatter = go.Mesh3d(y=df["longitude"],
                       x=df["latitude"], 
                       z=df['price_y'])#,
                       
                      # color=clusters.predict(X));
layout = go.Layout(title="Cluster of hosts",)
#                   scene=dict(xaxis=dict(title="lat"),
 #                             yaxis=dict(title="lon"),
  #                            zaxis=dict(title="price")));
fig = go.Figure(data=scatter, layout=layout);
plotly.offline.iplot(fig)